In [1]:
from pydantic import BaseModel, Field, field_validator, ValidationError

In [2]:
class Model(BaseModel):
    number: int = Field(gt=0, lt=10)

In [3]:
Model(number=2)

Model(number=2)

In [4]:
try:
    Model(number=13)
except ValidationError as ex:
    print(ex)

1 validation error for Model
number
  Input should be less than 10 [type=less_than, input_value=13, input_type=int]
    For further information visit https://errors.pydantic.dev/2.12/v/less_than


In [5]:
class Model(BaseModel):
    number: int = Field(gt=0, lt=10)

    @field_validator("number")
    @classmethod
    def validate_even(cls, value):
        print("Running custom validator")
        print(f"{value=}, {type(value)=}")
        return value


In [6]:
Model(number=3)

Running custom validator
value=3, type(value)=<class 'int'>


Model(number=3)

In [7]:
Model(number="3")

Running custom validator
value=3, type(value)=<class 'int'>


Model(number=3)

In [8]:
try:
    Model(number=12)
except ValidationError as ex:
    print(ex)

1 validation error for Model
number
  Input should be less than 10 [type=less_than, input_value=12, input_type=int]
    For further information visit https://errors.pydantic.dev/2.12/v/less_than


In [9]:
class Model(BaseModel):
    number: int = Field(gt=0, lt=10)

    @field_validator("number")
    @classmethod
    def validate_even(cls, value):
        print("Running custom validator")
        print(f"{value=}, {type(value)=}")
        if value % 2 == 0:    
            return value
        raise ValueError("Value must be even")


In [10]:
Model(number=4)

Running custom validator
value=4, type(value)=<class 'int'>


Model(number=4)

In [12]:
try:
    Model(number=5)
except ValidationError as ex:
    print(ex)

Running custom validator
value=5, type(value)=<class 'int'>
1 validation error for Model
number
  Value error, Value must be even [type=value_error, input_value=5, input_type=int]
    For further information visit https://errors.pydantic.dev/2.12/v/value_error


In [14]:
class Model(BaseModel):
    number: int = Field(gt=0, lt=10)

    @field_validator("number")
    @classmethod
    def validate_even(cls, value):
        if value % 2 == 1:
            return value + 1
        return value


In [15]:
Model(number=1)

Model(number=2)

In [16]:
from datetime import datetime
import pytz

def make_utc(dt: datetime) -> datetime:
    if dt.tzinfo is None:
        dt = pytz.utc.localize(dt)
    else:
        dt = dt.astimezone(pytz.utc)
    return dt


In [17]:
class Model(BaseModel):
    dt: datetime

    @field_validator("dt")
    @classmethod
    def make_utc(cls, dt: datetime) -> datetime:
        if dt.tzinfo is None:
            dt = pytz.utc.localize(dt)
        else:
            dt = dt.astimezone(pytz.utc)
        return dt


In [18]:
Model(dt="2025-01-01T03:00:00")

Model(dt=datetime.datetime(2025, 1, 1, 3, 0, tzinfo=<UTC>))

In [20]:
eastern = pytz.timezone("US/Eastern")
dt = eastern.localize(datetime(2020, 1, 1, 3, 0, 0))
dt

datetime.datetime(2020, 1, 1, 3, 0, tzinfo=<DstTzInfo 'US/Eastern' EST-1 day, 19:00:00 STD>)

In [21]:
Model(dt=dt)

Model(dt=datetime.datetime(2020, 1, 1, 8, 0, tzinfo=<UTC>))

In [22]:
class Model(BaseModel):
    number: int

    @field_validator("number")
    @classmethod
    def add_1(cls, value: int) -> int:
        print(f"Adding one to {value=}")
        return value + 1
    
    @field_validator("number")
    @classmethod
    def add_2(cls, value: int) -> int:
        print(f"Adding one to {value=}")
        return value + 2
    
    @field_validator("number")
    @classmethod
    def add_3(cls, value: int) -> int:
        print(f"Adding one to {value=}")
        return value + 3

In [23]:
Model(number=1)

Adding one to value=1
Adding one to value=2
Adding one to value=4


Model(number=7)

In [25]:
class Model(BaseModel):
    unit_cost: float
    unit_price: float

    @field_validator("unit_cost", "unit_price")
    @classmethod
    def round_2(cls, value: float) -> float:
        return round(value, 2)

In [27]:
Model(unit_cost=12.34233, unit_price=23.243234)

Model(unit_cost=12.34, unit_price=23.24)

In [28]:
class Model(BaseModel):
    unit_cost: float
    unit_price: float

    @field_validator("*")
    @classmethod
    def round_2(cls, value: float) -> float:
        return round(value, 2)

In [29]:
Model(unit_cost=12.34233, unit_price=23.243234)

Model(unit_cost=12.34, unit_price=23.24)